### 삼성 AI Expert 과정 알고리즘 강의 1일차 실습
### 3D Point Cloud Semantic Segmentation

1. confusion_matrix_to_metrics 함수 작성하기
2. MinkUNet18 모델 학습시키고 metric 확인하기


In [6]:
# link downloaded data
!ln -s /data /app/data

In [17]:
from typing import Dict, List, Optional, Tuple
import yaml

try:
    import hydra
    from hydra.core.config_store import ConfigStore
    from omegaconf import DictConfig, OmegaConf
except ImportError:
    print("Hydra and OmegaConf not installed, pip install hydra-core omegaconf")
    exit(1)

import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import warp as wp
from torch import Tensor
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Subset
from torchmetrics.classification import MulticlassConfusionMatrix
from tqdm import tqdm
from warpconvnet.dataset.scannet import ScanNetDataset
from warpconvnet.geometry.base.geometry import Geometry
from warpconvnet.geometry.types.points import Points
from warpconvnet.nn.modules.sparse_pool import PointToSparseWrapper

from mink_unet import MinkUNet18
from point_transformer_v3 import PointTransformerV3

In [52]:
# Embedded YAML configuration
CONFIG_YAML = """
# Path configuration
paths:
  data_dir: ./data/scannet_3d
  output_dir: ./results/
  ckpt_path: null

# Training configuration.
train:
  batch_size: 12
  lr: 0.001
  epochs: 50
  step_size: 20
  gamma: 0.7
  num_workers: 8

# Testing configuration
test:
  batch_size: 12
  num_workers: 4

# Dataset configuration
data:
  num_classes: 20
  voxel_size: 0.02
  ignore_index: 255

# Model configuration
model:
  _target_: mink_unet.MinkUNet18
  in_channels: 3
  out_channels: 20
  in_type: "voxel"

# General configuration
device: "cuda"
use_wandb: false
seed: 42
"""

In [18]:
def set_seed(seed: int):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def collate_fn(batch: List[Dict[str, Tensor]]):
    """
    Return dict of list of tensors
    """
    keys = batch[0].keys()
    return {key: [torch.tensor(item[key]) for item in batch] for key in keys}


class DataToTensor:
    def __init__(
        self,
        device: str = "cuda",
    ):
        self.device = device

    def __call__(self, batch_dict: Dict[str, Tensor]) -> Tuple[Geometry, Dict[str, Tensor]]:
        # cat all features into a single tensor
        cat_batch_dict = {k: torch.cat(v, dim=0).to(self.device) for k, v in batch_dict.items()}
        return (
            Points.from_list_of_coordinates(
                batch_dict["coords"],
                features=batch_dict["colors"],
            ).to(self.device),
            cat_batch_dict,
        )


def confusion_matrix_to_metrics(conf_matrix: Tensor) -> Dict[str, float]:
    """
    Return accuracy, miou, class_iou, class_accuracy

    Rows are ground truth, columns are predictions.
    """
    conf_matrix = conf_matrix.cpu()
    accuracy = (conf_matrix.diag().sum() / conf_matrix.sum()).item() * 100
    class_accuracy = (conf_matrix.diag() / conf_matrix.sum(dim=1)) * 100
    class_iou = conf_matrix.diag() / (
        conf_matrix.sum(dim=1) + conf_matrix.sum(dim=0) - conf_matrix.diag()
    )
    miou = class_iou.mean().item() * 100
    return {
        "accuracy": accuracy,
        "miou": miou,
        "class_iou": class_iou,
        "class_accuracy": class_accuracy,
    }

In [44]:
@torch.amp.autocast(device_type="cuda", enabled=True)
def train(
    model: nn.Module,
    train_loader: DataLoader,
    optimizer: optim.Optimizer,
    epoch: int,
    cfg: DictConfig,
):
    model.train()
    bar = tqdm(train_loader)
    dict_to_data = DataToTensor(device=cfg.device)
    for batch_dict in bar:
        start_time = time.time()
        optimizer.zero_grad()
        st, batch_dict = dict_to_data(batch_dict)
        output = model(st.to(cfg.device))
        loss = F.cross_entropy(
            output.features,
            batch_dict["labels"].long().to(cfg.device),
            reduction="mean",
            ignore_index=cfg.data.ignore_index,
        )
        loss.backward()
        optimizer.step()
        bar.set_description(f"Train Epoch: {epoch} Loss: {loss.item(): .3f}")


@torch.amp.autocast(device_type="cuda", enabled=True)
@torch.inference_mode()
def test(
    model: nn.Module,
    test_loader: DataLoader,
    cfg: DictConfig,
    num_test_batches: Optional[int] = None,
    save_visuals: bool = False,
):
    model.eval()
    torch.cuda.empty_cache()
    confusion_matrix = MulticlassConfusionMatrix(
        num_classes=cfg.data.num_classes, ignore_index=cfg.data.ignore_index
    ).to(cfg.device)
    test_loss = 0
    num_batches = 0

    visual_data = []
    dict_to_data = DataToTensor(device=cfg.device)
    for batch_dict in tqdm(test_loader):
        original_batch_dict = batch_dict
        st, batch_dict = dict_to_data(batch_dict)
        output = model(st.to(cfg.device))
        labels = batch_dict["labels"].long().to(cfg.device)
        test_loss += F.cross_entropy(
            output.features,
            labels,
            reduction="mean",
            ignore_index=cfg.data.ignore_index,
        ).item()
        pred = output.features.argmax(dim=1)
        confusion_matrix.update(pred, labels)

        if save_visuals:
            num_items_in_batch = len(st.offsets) - 1
            for i in range(num_items_in_batch):
                start_idx = st.offsets[i]
                end_idx = st.offsets[i+1]

                visual_data.append({
                    "coords": original_batch_dict["coords"][i],
                    "colors": original_batch_dict["colors"][i],
                    "preds": pred[start_idx:end_idx].cpu(),
                    "labels": labels[start_idx:end_idx].cpu(),
                })
        
        num_batches += 1
        if num_test_batches is not None and num_batches >= num_test_batches:
            break

    if save_visuals and visual_data:
        save_path = cfg.paths.output_dir + "visual_predictions.pt"
        os.makedirs(cfg.paths.output_dir, exist_ok=True)
        torch.save(visual_data, save_path)
        print(f"\nSaved visualization data for {len(visual_data)} point clouds to {save_path}")

    metrics = confusion_matrix_to_metrics(confusion_matrix.compute())
    
    print(
        f"\nTest set: Average loss: {test_loss / num_batches: .4f}, Accuracy: {metrics['accuracy']: .2f}%, mIoU: {metrics['miou']: .2f}%\n"
    )
    return metrics


In [53]:
##### MAIN BLOCK #####

# load YAML
cfg_dict = yaml.safe_load(CONFIG_YAML)
cfg = OmegaConf.create(cfg_dict)

# Print config
print(OmegaConf.to_yaml(cfg))

# Initialize seed
set_seed(cfg.seed)

device = torch.device(cfg.device)

train_dataset = ScanNetDataset(cfg.paths.data_dir, split="train",)
train_dataset = Subset(train_dataset, range(100))
test_dataset = ScanNetDataset(cfg.paths.data_dir, split="val",)
test_dataset = Subset(test_dataset, range(50))

train_loader = DataLoader(
    train_dataset,
    batch_size=cfg.train.batch_size,
    num_workers=cfg.train.num_workers,
    shuffle=True,
    collate_fn=collate_fn,
)
test_loader = DataLoader(
    test_dataset,
    batch_size=cfg.test.batch_size,
    num_workers=cfg.test.num_workers,
    shuffle=False,
    collate_fn=collate_fn,
)

# Model initialization
model = MinkUNet18(
    in_channels=cfg.model.in_channels,
    out_channels=cfg.model.out_channels,
).to(device)

if hasattr(cfg.model, "in_type") and cfg.model.in_type == "voxel":
    model = PointToSparseWrapper(
        inner_module=model,
        voxel_size=cfg.data.voxel_size,
        concat_unpooled_pc=False,
    )

optimizer = optim.AdamW(model.parameters(), lr=cfg.train.lr)
scheduler = StepLR(optimizer, step_size=cfg.train.step_size, gamma=cfg.train.gamma)

# Test before training
metrics = test(model, test_loader, cfg, num_test_batches=5)

for epoch in range(1, cfg.train.epochs + 1):
    train(
        model,
        train_loader,
        optimizer,
        epoch,
        cfg,
    )
    should_save_visuals = (epoch == cfg.train.epochs)
    metrics = test(model, test_loader, cfg, save_visuals=should_save_visuals)
    scheduler.step()

print(f"Final mIoU: {metrics['miou']: .2f}%")

paths:
  data_dir: ./data/scannet_3d
  output_dir: ./results/
  ckpt_path: null
train:
  batch_size: 12
  lr: 0.001
  epochs: 50
  step_size: 20
  gamma: 0.7
  num_workers: 8
test:
  batch_size: 12
  num_workers: 4
data:
  num_classes: 20
  voxel_size: 0.02
  ignore_index: 255
model:
  _target_: mink_unet.MinkUNet18
  in_channels: 3
  out_channels: 20
  in_type: voxel
device: cuda
use_wandb: false
seed: 42



 80%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                             | 4/5 [00:02<00:00,  1.34it/s]



Test set: Average loss:  2.9742, Accuracy:  2.69%, mIoU:  0.40%



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.65it/s]



Test set: Average loss:  2.3006, Accuracy:  51.35%, mIoU:  5.30%



...


Test set: Average loss:  1.8332, Accuracy:  53.36%, mIoU:  5.89%



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.66it/s]



Test set: Average loss:  2.0276, Accuracy:  52.01%, mIoU:  6.62%



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.66it/s]



Saved visualization data for 50 point clouds to ./results/visual_predictions.pt

Test set: Average loss:  2.5116, Accuracy:  48.17%, mIoU:  6.77%

Final mIoU:  6.77%
